In [ ]:
!pip install keras==2.2.0

In [ ]:

from __future__ import print_function, division

from keras.applications.vgg19 import VGG19
from keras.layers import Input, Dense, Flatten, Dropout, Reshape
from keras.layers import BatchNormalization, Activation, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.optimizers import Adam
from keras import models
from keras import layers
from keras import optimizers

from keras.datasets import cifar10
import keras.backend as K
import tensorflow as tf
from tensorflow.python.keras import backend as K
from keras.models import load_model

import matplotlib.pyplot as plt

import sys
import numpy as np

%pylab inline

In [ ]:
!pip install keras_vggface

In [ ]:
from keras_applications.imagenet_utils import _obtain_input_shape

In [ ]:
from keras_vggface import VGGFace

In [ ]:
def get_generator(input_layer):
 
  hid = Dense(128 * 32 * 32, activation='relu')(input_layer)    
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
  hid = Reshape((32, 32, 128))(hid)

  hid = Conv2D(128, kernel_size=5, strides=1,padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)    
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2DTranspose(128, 4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2DTranspose(128, 4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
                      
  hid = Conv2D(3, kernel_size=5, strides=1, padding="same")(hid)
  out = Activation("tanh")(hid)
  model = Model(input_layer, out)
  model.summary()
  
  return model, out

In [ ]:
def get_discriminator(input_layer):
  '''
  Requires the input layer as input, outputs the model and the final layer
  '''
  vgg_model = VGGFace(include_top=False, input_shape=(128, 128, 3))
  nb_class = 1
  hidden_dim = 512
  for layer in vgg_model.layers[:9]:
    layer.trainable=False
  last_layer = vgg_model.output
  x = Flatten(name='flatten')(last_layer)
  x = Dense(hidden_dim,name='fc6')(x)
  x = LeakyReLU(alpha=0.1)(x)
  #x = Dense(128,name='fc7')(x)
  #x = LeakyReLU(alpha=0.1)(x)
  out = Dense(nb_class, activation='sigmoid', name='fc8')(x)  

  model = Model(vgg_model.input, out)

  model.summary()

  return model, out

In [ ]:
from keras.preprocessing import image

def generate_noise(n_samples, noise_dim):
  X = np.random.normal(0, 1, size=(n_samples, noise_dim))
  return X

def show_imgs(batchidx):
  noise = generate_noise(9, 100)
  gen_imgs = generator.predict(noise)

  fig, axs = plt.subplots(3, 3)
  count = 0
  for i in range(3):
    for j in range(3):
      img = image.array_to_img(gen_imgs[count], scale=True)
      axs[i,j].imshow(img)
      axs[i,j].axis('off')
      count += 1
  plt.show()
  plt.close()

In [ ]:
# GAN creation
img_input = Input(shape=(128,128,3))
discriminator, disc_out = get_discriminator(img_input)
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False

noise_input = Input(shape=(100,))
generator, gen_out = get_generator(noise_input)

gan_input = Input(shape=(100,))
x = generator(gan_input)
gan_out = discriminator(x)
gan = Model(gan_input, gan_out)
gan.summary()

gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

In [ ]:
import os

In [ ]:
# Data
BASE_PATH = "../input/celebrities-100k/"
print(os.listdir(BASE_PATH))
#BASE_PATH = base_PATH+ 'celebrities-100k' 
DATASET_LIST_PATH= BASE_PATH + "100k.txt"
print(DATASET_LIST_PATH)

In [ ]:
INPUT_DATA_DIR = BASE_PATH + "100k/100k/"

In [ ]:
OUTPUT_DIR = "./"

In [ ]:
#MODEL_PATH = BASE_PATH + "models/" + "model_" + str(EPOCH) + ".ckpt"
DATASET = [INPUT_DATA_DIR + str(line).rstrip() for line in open(DATASET_LIST_PATH,"r")]

In [ ]:
len(DATASET)

In [ ]:
data=DATASET[:10000]

In [ ]:
import scipy.misc
import scipy.ndimage

In [ ]:
#uploading images from folder
images = []
for img_name in data:
    image = scipy.ndimage.imread(img_name, mode="RGB")
    image_resized = scipy.misc.imresize(image,(128,128))
    images.append(image_resized)

In [ ]:
train_x = np.stack(images)

In [ ]:
X_train = train_x

In [ ]:
X_train = ((X_train)-127.5)/127.5

In [ ]:

BATCH_SIZE = 64

# # Get training images
#(X_train, y_train), (X_test, _) = cifar10.load_data()

# S

# Normalize data
 
num_batches = int(X_train.shape[0]/BATCH_SIZE)

In [ ]:
X_train.shape

In [ ]:
import time

In [ ]:
N_EPOCHS = 200
CUMM_DLOSS=0
for epoch in range(N_EPOCHS):
  tic=time.time()
  cum_d_loss = 0.
  cum_g_loss = 0.
  
  for batch_idx in range(num_batches):
    # Get the next set of real images to be used in this iteration
    images = X_train[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]

    noise_data = generate_noise(BATCH_SIZE, 100)
    generated_images = generator.predict(noise_data)

    # Train on soft labels (add noise to labels as well)
    noise_prop = 0.05 # Randomly flip 5% of labels
    
    # Prepare labels for real data
    true_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
    true_labels[flipped_idx] = 1 - true_labels[flipped_idx]
    
    # Train discriminator on real data
    d_loss_true = discriminator.train_on_batch(images, true_labels)

    # Prepare labels for generated data
    gene_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
    gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]
    # Train discriminator on generated data
    d_loss_gene = discriminator.train_on_batch(generated_images, gene_labels)

    d_loss = 0.5*np.add(d_loss_true, d_loss_gene)
    cum_d_loss += d_loss
    # Train generator
    noise_data = generate_noise(BATCH_SIZE, 100)
    g_loss = gan.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
    cum_g_loss += g_loss

  #for whole data:-
  IMAGES=X_train 
  Noise_Data = generate_noise(10000, 100)
  Generated_Images = generator.predict(Noise_Data)
  # Prepare labels for real data
  True_Labels = np.zeros((10000, 1)) + np.random.uniform(low=0.0, high=0.1, size=(10000, 1))
  Flipped_Idx = np.random.choice(np.arange(len(True_Labels)), size=int(noise_prop*len(True_Labels)))
  True_Labels[Flipped_Idx] = 1 - True_Labels[Flipped_Idx]
  
  # EVALUATE discriminator on real data
  DLOSS_TRUE = discriminator.evaluate(IMAGES, True_Labels) 

  # Prepare labels for generated data
  Gene_Labels = np.ones((10000, 1)) - np.random.uniform(low=0.0, high=0.1, size=(10000, 1))
  Flipped_Idx = np.random.choice(np.arange(len(Gene_Labels)), size=int(noise_prop*len(Gene_Labels)))
  Gene_Labels[Flipped_Idx] = 1 - Gene_Labels[Flipped_Idx]
  # EVALUTE discriminator on generated data
  D_LOSS_GENE = discriminator.evaluate(Generated_Images, Gene_Labels) 
  D_LOSS=0.5*np.add(DLOSS_TRUE,D_LOSS_GENE)
  CUMM_DLOSS+=D_LOSS
   
  print("  Epoch:- "+str(epoch+1)+", D LOSS: "+str(D_LOSS) + "; Cummulative D_LOSS: "+ str(CUMM_DLOSS) )
    
  
  noise_data = generate_noise(BATCH_SIZE, 100)
  #generated_images = generator.predict(noise_data)
  print('  Epoch: {}, Generator Loss: {}, Discriminator Loss: {}'.format(epoch+1, cum_g_loss/num_batches, cum_d_loss/num_batches))
  show_imgs("epoch" + str(epoch))
  toc=time.time()
  print("For epoch: "+ str(epoch +1)+", Time taken is: " +str(toc-tic)+ " seconds")
  #print(discriminator.predict(generated_images))